# import Wallonia's Q observations
```
file format
how to acquire
reformat
saved format
```

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import re
%matplotlib inline


In [19]:
DATAPATH = "meuse_forecast/datasets/wallonie/"
os.chdir("/home/paperspace")
os.chdir(DATAPATH)
all_stations_csv = glob.glob("*.csv")

In [48]:
# iterate over all csv files (one for every station)
for csv in sorted(all_stations_csv):
    station_code, observation_code, freq_code = re.match("^(\d\d\d\d)(\d\d\d\d)_(...)\.csv", csv).groups()
    print(station_code, observation_code, freq_code)

    # read csv into pandas: 
    #  separator contains silly space after semicolon\
    #  column 'date' becomes the index (string/object type)
    #  we skip line #0, even though it contains interesting meta info
    raw_df = pd.read_csv(csv, sep=';\s?', engine='python', index_col=['date'], header=1 )
    
    # drop the last, unnamed, column which is the result of a spurious, trailing separator
    raw_df.drop(labels=['Unnamed: 25'], axis=1, inplace=True)
    
    # break if you find any NaNs
    assert raw_df.isna().sum().sum() == 0, "NaNs detected in {}".format(station_code)
    
    # reconstruct object typed index into DateTimeIndex according to Pandas' law
    raw_df.index = pd.to_datetime(raw_df.index)
    
    # construct an index like it, but guaranteed to have all days in it
    complete_index = pd.DatetimeIndex(start=raw_df.index[0], end=raw_df.index[-1], freq='D')
    
    # create an idx into index that points out the days that should be in raw_df, but are not (~)
    missing_days_idx = ~ complete_index.isin(raw_df.index)
    # create a new, shorter, index containing all the missing days (pull them from complete_index, masked by idx)
    missing_index = complete_index[missing_days_idx]
    n_missing = len(missing_index)
    
    # Only if we are missing days in our index,
    if n_missing > 0:
        print(station_code, "is missing", n_missing, "days: filling up with NaN rows")
        # append missing index to raw_df (creating empty rows), then sort again
        raw_df = raw_df.append(pd.DataFrame(index=missing_index))
        raw_df.sort_index(inplace=True)
    
    # break if the given index differs from the perfect index
    assert raw_df.index[0] == complete_index[0]   , "index still starts funny in {}".format(station_code)
    assert raw_df.index[-1] == complete_index[-1] , "index still ends funny in {}".format(station_code)

    # now construct an hourly index, based on the start/end of the complete index
    hourly_index = pd.DatetimeIndex(start=complete_index[0], periods=24*complete_index.shape[0], freq='H')
    
    # create a clean DataFrame, using the hourly index, with one empty column named 'Q'
    clean_df = pd.DataFrame(index=hourly_index, columns=['Q'])
    clean_df.index.name = "date_time"
    
    start = clean_df.index[0]
    end = clean_df.index[-1]
    start_year = start.year
    end_year = end.year
    
    # dump all numbers from the old, matrix-shaped, raw_df directly into the new, colum-shaped, clean_df
    clean_df['Q'] = raw_df.values.ravel()
    
    # final check to see if the final elements in either df match
    assert clean_df['Q'][-1] == raw_df.iloc[-1, -1] , "final elements do not match in {}".format(station_code)
    
    filename = "Q_{}_hourly_{}-{}.csv".format(station_code, start.year, end.year)
    print(" station {} has Q from year {} until {}".format(station_code, start.year, end.year))
    print(" saving to", filename)
    clean_df.to_csv(filename)
#


5291 1002 hor
 station 5291 has Q from year 2009 until 2016
 saving to Q_5291_hourly_2009-2016.csv
5572 1002 hor
 station 5572 has Q from year 1998 until 2016
 saving to Q_5572_hourly_1998-2016.csv
5826 1002 hor
 station 5826 has Q from year 1987 until 2017
 saving to Q_5826_hourly_1987-2017.csv
5921 1002 hor
 station 5921 has Q from year 1988 until 2016
 saving to Q_5921_hourly_1988-2016.csv
5953 1002 hor
 station 5953 has Q from year 1997 until 2016
 saving to Q_5953_hourly_1997-2016.csv
5962 1002 hor
 station 5962 has Q from year 1994 until 2016
 saving to Q_5962_hourly_1994-2016.csv
5991 1002 hor
 station 5991 has Q from year 1978 until 2016
 saving to Q_5991_hourly_1978-2016.csv
6021 1002 hor
 station 6021 has Q from year 1978 until 2016
 saving to Q_6021_hourly_1978-2016.csv
6122 1002 hor
 station 6122 has Q from year 1982 until 2016
 saving to Q_6122_hourly_1982-2016.csv
6228 1002 hor
 station 6228 has Q from year 1992 until 2017
 saving to Q_6228_hourly_1992-2017.csv
6526 1002 

In [171]:
raw_df = pd.read_csv("datasets/wallonie/74871002_hor.csv", sep=';\s?', engine='python', index_col=['date'], header=1 )

In [172]:
raw_df.drop(labels=['Unnamed: 25'], axis=1, inplace=True)

In [173]:
raw_df.head()

,heure1,heure2,heure3,heure4,heure5,heure6,heure7,heure8,heure9,heure10,...,heure15,heure16,heure17,heure18,heure19,heure20,heure21,heure22,heure23,heure24
date,,,,,,,,,,,,,,,,,,,,,
1998-01-01,22.15,22.20,22.65,23.62,25.14,26.67,28.35,29.44,30.18,31.83,...,32.38,32.19,32.31,32.51,32.59,32.68,32.58,32.43,31.96,30.93
1998-01-02,29.59,28.80,28.40,28.75,29.15,29.37,29.48,29.38,29.40,31.65,...,34.80,40.16,40.59,43.96,42.03,42.63,44.46,45.92,48.12,52.82
1998-01-03,54.86,55.19,55.61,55.99,56.22,56.62,56.77,57.32,58.25,63.28,...,63.84,64.35,64.29,64.15,64.17,64.12,64.05,63.93,63.94,64.18
1998-01-04,63.99,64.04,64.17,64.19,64.13,64.12,63.96,63.80,63.88,64.02,...,64.11,64.67,65.60,65.28,64.76,64.51,64.36,64.10,63.99,63.88
1998-01-05,64.08,64.06,64.21,64.44,64.57,64.54,64.49,64.19,64.12,63.86,...,62.67,62.33,62.44,62.56,62.33,62.20,61.89,61.51,61.21,61.10


In [174]:
raw_df.tail()

,heure1,heure2,heure3,heure4,heure5,heure6,heure7,heure8,heure9,heure10,...,heure15,heure16,heure17,heure18,heure19,heure20,heure21,heure22,heure23,heure24
date,,,,,,,,,,,,,,,,,,,,,
2018-03-27,9.263,9.552,9.972,10.108,10.106,10.170,10.008,9.764,9.498,9.194,...,7.993,8.054,8.167,8.253,8.348,8.640,9.036,9.330,9.360,9.305
2018-03-28,9.223,9.138,9.071,8.982,8.944,8.933,8.901,9.081,8.921,9.008,...,14.587,15.073,16.264,18.333,18.892,18.168,18.342,18.330,17.907,17.273
2018-03-29,16.692,15.877,15.255,14.821,14.653,14.558,14.561,14.716,14.782,15.027,...,18.289,18.580,18.798,19.216,19.640,20.271,20.101,19.948,19.553,20.087
2018-03-30,20.672,20.624,20.095,19.053,16.813,14.719,13.501,13.323,13.659,14.111,...,16.611,16.636,16.622,16.701,16.934,16.932,16.796,16.774,16.655,16.503
2018-03-31,16.382,16.224,16.197,16.172,16.229,16.254,16.301,16.253,16.119,16.057,...,13.445,12.356,11.294,11.384,13.906,17.245,19.401,19.769,19.490,19.089


In [175]:
assert raw_df.isna().sum().sum() == 0

In [176]:
raw_df.columns

Index(['heure1', 'heure2', 'heure3', 'heure4', 'heure5', 'heure6', 'heure7',
       'heure8', 'heure9', 'heure10', 'heure11', 'heure12', 'heure13',
       'heure14', 'heure15', 'heure16', 'heure17', 'heure18', 'heure19',
       'heure20', 'heure21', 'heure22', 'heure23', 'heure24'],
      dtype='object')

In [177]:
print( raw_df.loc["1998-01-01", "heure24"] )

30.93


In [178]:
raw_df.index = pd.to_datetime(raw_df.index)

In [179]:
given_index = raw_df.index
given_index

DatetimeIndex(['1998-01-01', '1998-01-02', '1998-01-03', '1998-01-04',
               '1998-01-05', '1998-01-06', '1998-01-07', '1998-01-08',
               '1998-01-09', '1998-01-10',
               ...
               '2018-03-22', '2018-03-23', '2018-03-24', '2018-03-25',
               '2018-03-26', '2018-03-27', '2018-03-28', '2018-03-29',
               '2018-03-30', '2018-03-31'],
              dtype='datetime64[ns]', name='date', length=7371, freq=None)

In [180]:
complete_index = pd.DatetimeIndex(start=given_index[0], end=given_index[-1], freq='D')

In [181]:
complete_index

DatetimeIndex(['1998-01-01', '1998-01-02', '1998-01-03', '1998-01-04',
               '1998-01-05', '1998-01-06', '1998-01-07', '1998-01-08',
               '1998-01-09', '1998-01-10',
               ...
               '2018-03-22', '2018-03-23', '2018-03-24', '2018-03-25',
               '2018-03-26', '2018-03-27', '2018-03-28', '2018-03-29',
               '2018-03-30', '2018-03-31'],
              dtype='datetime64[ns]', length=7395, freq='D')

In [182]:
missing_days_idx = ~ complete_index.isin(given_index)

missing_index = complete_index[missing_days_idx]

In [184]:
missing_index

DatetimeIndex(['2016-10-16', '2016-10-17', '2016-10-18', '2016-10-19',
               '2016-10-20', '2016-10-21', '2016-10-22', '2016-10-23',
               '2016-10-24', '2016-10-25', '2016-10-26', '2016-10-27',
               '2016-10-28', '2016-10-29', '2016-10-30', '2016-10-31',
               '2016-11-01', '2016-11-02', '2016-11-03', '2016-11-04',
               '2016-11-05', '2016-11-06', '2016-11-07', '2016-11-08'],
              dtype='datetime64[ns]', freq='D')

In [185]:
len(missing_index)

24

In [186]:
stuffed_df = raw_df.append(pd.DataFrame(index=missing_index))

In [187]:
stuffed_df.describe()

,heure1,heure10,heure11,heure12,heure13,heure14,heure15,heure16,heure17,heure18,...,heure22,heure23,heure24,heure3,heure4,heure5,heure6,heure7,heure8,heure9
count,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,...,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000,7371.000000
mean,10.838927,3.926603,5.290557,5.323077,2.635353,4.001325,5.350382,3.975526,3.960523,3.937757,...,5.385560,6.769341,9.491799,8.078636,8.065657,6.701276,5.334310,3.968334,3.944827,3.932972
std,165.802131,308.898722,286.073744,286.072013,330.136309,308.893337,286.070720,308.893154,308.894169,308.895185,...,286.079657,261.264950,202.691473,233.821633,233.820637,261.261032,286.075587,308.897200,308.897749,308.898759
min,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
25%,3.501500,3.482000,3.539500,3.603500,3.681500,3.738500,3.730000,3.731500,3.720000,3.690000,...,3.511500,3.510000,3.490000,3.430000,3.440000,3.430000,3.460000,3.478500,3.450000,3.460000
50%,6.790000,6.770000,6.843000,6.910000,6.960000,6.970000,6.940000,6.870000,6.780000,6.682000,...,6.750000,6.780000,6.780000,6.750000,6.720000,6.730000,6.730000,6.710000,6.670000,6.699000
75%,15.660000,15.145000,15.115000,15.054000,15.000000,14.940000,14.965000,14.825000,15.060500,14.884500,...,15.358000,15.459000,15.610000,15.641500,15.590000,15.595000,15.485000,15.510000,15.370000,15.224000
max,130.607000,135.321000,135.002000,134.448000,133.940000,133.900000,133.630000,133.270000,133.320000,132.620000,...,131.760000,131.440000,130.740000,133.433000,134.202000,134.571000,134.803000,135.083000,135.354000,135.451000


In [188]:
stuffed_df.isna().sum()

heure1     24
heure10    24
heure11    24
heure12    24
heure13    24
heure14    24
heure15    24
heure16    24
heure17    24
heure18    24
heure19    24
heure2     24
heure20    24
heure21    24
heure22    24
heure23    24
heure24    24
heure3     24
heure4     24
heure5     24
heure6     24
heure7     24
heure8     24
heure9     24
dtype: int64

In [189]:
raw_df.index

DatetimeIndex(['1998-01-01', '1998-01-02', '1998-01-03', '1998-01-04',
               '1998-01-05', '1998-01-06', '1998-01-07', '1998-01-08',
               '1998-01-09', '1998-01-10',
               ...
               '2018-03-22', '2018-03-23', '2018-03-24', '2018-03-25',
               '2018-03-26', '2018-03-27', '2018-03-28', '2018-03-29',
               '2018-03-30', '2018-03-31'],
              dtype='datetime64[ns]', name='date', length=7371, freq=None)

In [190]:
stuffed_df.sort_index(inplace=True)

In [191]:
stuffed_df.tail()

,heure1,heure10,heure11,heure12,heure13,heure14,heure15,heure16,heure17,heure18,...,heure22,heure23,heure24,heure3,heure4,heure5,heure6,heure7,heure8,heure9
2018-03-27,9.263,9.194,8.917,8.711,8.427,8.109,7.993,8.054,8.167,8.253,...,9.330,9.360,9.305,9.972,10.108,10.106,10.170,10.008,9.764,9.498
2018-03-28,9.223,9.008,11.557,11.939,12.839,13.854,14.587,15.073,16.264,18.333,...,18.330,17.907,17.273,9.071,8.982,8.944,8.933,8.901,9.081,8.921
2018-03-29,16.692,15.027,15.347,16.090,16.761,17.501,18.289,18.580,18.798,19.216,...,19.948,19.553,20.087,15.255,14.821,14.653,14.558,14.561,14.716,14.782
2018-03-30,20.672,14.111,14.864,15.760,16.405,16.653,16.611,16.636,16.622,16.701,...,16.774,16.655,16.503,20.095,19.053,16.813,14.719,13.501,13.323,13.659
2018-03-31,16.382,16.057,16.029,15.976,15.231,14.284,13.445,12.356,11.294,11.384,...,19.769,19.490,19.089,16.197,16.172,16.229,16.254,16.301,16.253,16.119


In [53]:
assert given_index[0] == complete_index[0]
assert given_index[-1] == complete_index[-1]


AssertionError: 

In [54]:
print(given_index[-1] , complete_index[-1])

2018-03-31 00:00:00 2018-03-07 00:00:00


In [15]:
hourly_index = pd.DatetimeIndex(start=complete_index[0], periods=24*complete_index.shape[0], freq='H')

In [16]:
hourly_index

DatetimeIndex(['1988-01-01 00:00:00', '1988-01-01 01:00:00',
               '1988-01-01 02:00:00', '1988-01-01 03:00:00',
               '1988-01-01 04:00:00', '1988-01-01 05:00:00',
               '1988-01-01 06:00:00', '1988-01-01 07:00:00',
               '1988-01-01 08:00:00', '1988-01-01 09:00:00',
               ...
               '2016-08-31 14:00:00', '2016-08-31 15:00:00',
               '2016-08-31 16:00:00', '2016-08-31 17:00:00',
               '2016-08-31 18:00:00', '2016-08-31 19:00:00',
               '2016-08-31 20:00:00', '2016-08-31 21:00:00',
               '2016-08-31 22:00:00', '2016-08-31 23:00:00'],
              dtype='datetime64[ns]', length=251304, freq='H')

In [17]:
clean_df = pd.DataFrame(index=hourly_index, columns=['Q'])

In [18]:
clean_df['Q'] = raw_df.values.ravel()

In [19]:
clean_df.describe()

,Q
count,251304.000000
mean,22.807982
std,27.964947
min,0.796000
25%,5.827000
50%,12.776000
75%,29.466000
max,432.991000


In [20]:
clean_df.head()

,Q
1988-01-01 00:00:00,22.273
1988-01-01 01:00:00,22.225
1988-01-01 02:00:00,22.225
1988-01-01 03:00:00,22.557
1988-01-01 04:00:00,22.557


In [21]:
clean_df.tail()

,Q
2016-08-31 19:00:00,4.933
2016-08-31 20:00:00,4.931
2016-08-31 21:00:00,4.927
2016-08-31 22:00:00,4.913
2016-08-31 23:00:00,4.910


In [24]:
assert clean_df['Q'][-1] == raw_df.iloc[-1, -1]